# Featurestore - Churn Demo
## Feature Store Setup
Christos Aniftos

## Install required libraries

## Import modules

In [ ]:
from google.cloud import aiplatform

In [ ]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import FeaturestoreMonitoringConfig
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [ ]:
PROJECT_ID=!gcloud config get-value project # returns default project id 
PROJECT_ID=PROJECT_ID[0]

LOCATION = 'europe-west4' 
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "telco"
ENTITY="phone"

## Define clients for FS admin and data management

In [ ]:
# Create admin_client for CRUD
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [ ]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

## Create FeatureStore

In [ ]:
new_feature_store = featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1 # The number of nodes for each cluster (bigtable)
            )
        )
try:
    print(
        admin_client.create_featurestore(
            featurestore_service_pb2.CreateFeaturestoreRequest(
                parent=LOC_PATH,
                featurestore_id=FEATURESTORE_ID,
                featurestore= new_feature_store
            )
        ).result()
    )
except Exception as ex:
    print(ex)

## Create a feature store Entity

In [ ]:
try:
    entity_type_obj = admin_client.create_entity_type(
        featurestore_service_pb2.CreateEntityTypeRequest(
            parent=FS_PATH,
            entity_type_id=ENTITY, 
            entity_type=entity_type_pb2.EntityType(description="phone features"))).result()
    print(entity_type_obj)
except Exception as ex:
    print(ex)

In [ ]:
from google.protobuf import duration_pb2 as duration 

duration = duration.Duration()
duration.FromSeconds(seconds=60)


fmc=FeaturestoreMonitoringConfig(
    snapshot_analysis = FeaturestoreMonitoringConfig.SnapshotAnalysis(disabled=False, monitoring_interval = duration))
fmc

In [ ]:
entity_type_obj.monitoring_config= fmc
entity_type_obj = admin_client.update_entity_type(featurestore_service_pb2.UpdateEntityTypeRequest(
            entity_type=entity_type_pb2.EntityType(name=entity_type_obj.name)),
            )

## Create features in batch

In [ ]:
features_list_tmp = []

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.INT64,
                    description="phone price euro",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="approx_price_euro"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="phone brand",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="brand"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="phone model",
                    labels=[("pii", ""),("approved", "")]),
                feature_id="model"))
    
try:
    features_list = admin_client.batch_create_features(
        parent=ENTITY_PATH,
        requests=features_list_tmp).result()
    print(features_list)
except Exception as ex:
    print(ex)